<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="4" color="black"><b>Use the Python Function feature to scrape a webpage</b></font></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/pmservice/wml-sample-notebooks/master/images/python.png?raw=true" width="600" alt="Icon"> </th>
   </tr>
</table>

A Python Function is a feature to save and deploy Python code through notebooks or IDE. Python Functions can be implemented in Python notebooks or through REST API using IDE.

The requirement of a Python Function is to have a `score()` function inside the Python Function. This `score()` function will be called when running the deployed Python Function.

A Python Function can be:
- Saved in the Watson Machine Learning (WML) repository.
- Deployed in the Watson Machine Learning (WML) repository.
- Scored.


**Note**: For more details on Python Function, please refer to this <a href="https://dataplatform.cloud.ibm.com/docs/content/analyze-data/ml-deploy-functions.html?audience=wdp" target="_blank" rel="noopener no referrer">link</a>.

In this notebook, saving, deploying, and scoring a Python Function will be demonstrated. Although the `score()` function is intended to score a Python Function, it can also have other custom functionality such as preprocessing texts.

The `score()` function of the Python Function that will be presented in this example notebook does the following tasks:
- Scrapes texts that are enclosed in `<p>` tags.
- Tokenizes scraped texts.
    
The data that will be used in this notebook is the <a href="http://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection" target="_blank" rel="noopener no referrer">SMS spam data set</a> from the UCI Machine Learning Repository. 

The original data set has both texts and labels in a single file. Only the text parts of the data set were extracted and converted into an `html` file.

You can find the html version of the SMS messages <a href="https://github.com/pmservice/wml-sample-notebooks/tree/master/datasets" target="_blank" rel="noopener no referrer">here</a>.

## Contents

This notebook contains the following parts:

1.	[Define a Python Function](#function)
2.	[Save, deploy, and score the Python Function](#deploy)
3.	[Summary and next steps](#summary)

## 1. Define a Python Function <a id="function"></a>

You can pass a `parameter dict` to the Python Function like in the cell below.

In [1]:
# You can add any information needed to run the Python function, e.g., wml credentials
py_params = {

}

The code outside the `score()` function only executes one time and can do things like load objects, install libs, etc. 

In this example, the `score()` function takes the `url`(s) of the payload and passes it (them) to BeautifulSoup to scrape texts enclosed in `<p>` tags. The extracted texts are passed to `scikit-learn`'s CountVectorizer in order to tokenize the texts.

**Note**: If you are importing modules inside the Python Function, you have to install packages through the `subprocess` module. More details can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/analyze-data/ml-functions.html?audience=wdp#import" target="_blank" rel="noopener no referrer">here</a> in the documentation for Python Function.

In [2]:
def py_funct(params=py_params):  
    def score(payload):
        # Import the subprocess module.
        import subprocess
        
        # Install required packages.
        subprocess.check_output('pip install --user lxml', stderr=subprocess.STDOUT, shell=True)
        subprocess.check_output('pip install --user bs4', stderr=subprocess.STDOUT, shell=True)
        subprocess.check_output('pip install --user sklearn', stderr=subprocess.STDOUT, shell=True)
        
        # Import required modules.
        from bs4 import BeautifulSoup
        from urllib.request import urlopen
        from sklearn.feature_extraction.text import CountVectorizer
        
        urls = payload['values']
        final_texts = []   # An array that will have stripped clean text from html tag enclosed text.
        
        for url in urls:            
            html = urlopen(url)
            soup = BeautifulSoup(html, 'lxml')

            p_tags = soup.find_all('p')    # Text is enclosed in <p> tag.

            for p in p_tags:
                str_p = str(p)
                text = BeautifulSoup(str_p, 'lxml').get_text()
                final_texts.append(text)
        
        vectorizer = CountVectorizer()
        vectorizer.fit_transform(final_texts)
        
        return {'tokens': vectorizer.get_feature_names()}
        
    return score

Prepare a sample payload like in the following cell.

In [3]:
sample_data = {
    'fields': ['url'],
    'values': [
        'https://raw.githubusercontent.com/pmservice/wml-sample-notebooks/master/datasets/sms_spam_text.html'
    ]
}

Pass the list of `url`s to the Python Function.

In [4]:
pf = py_funct(py_params)
tokens = pf(sample_data)

The Python Function object returns a `dict` that has a list of tokens as the `value` - the name of the `value` is `tokens`.

In [5]:
# Token list
tokens['tokens']

['00',
 '000',
 '000pes',
 '008704050406',
 '0089',
 '0121',
 '01223585236',
 '01223585334',
 '0125698789',
 '02',
 '0207',
 '02072069400',
 '02073162414',
 '02085076972',
 '021',
 '03',
 '04',
 '0430',
 '05',
 '050703',
 '0578',
 '06',
 '07',
 '07008009200',
 '07046744435',
 '07090201529',
 '07090298926',
 '07099833605',
 '07123456789',
 '0721072',
 '07732584351',
 '07734396839',
 '07742676969',
 '07753741225',
 '0776xxxxxxx',
 '07781482378',
 '07786200117',
 '077xxx',
 '078',
 '07801543489',
 '07808',
 '07808247860',
 '07808726822',
 '07815296484',
 '07821230901',
 '078498',
 '07880867867',
 '0789xxxxxxx',
 '07946746291',
 '0796xxxxxx',
 '07973788240',
 '07xxxxxxxxx',
 '08',
 '0800',
 '08000407165',
 '08000776320',
 '08000839402',
 '08000930705',
 '08000938767',
 '08001950382',
 '08002888812',
 '08002986030',
 '08002986906',
 '08002988890',
 '08006344447',
 '0808',
 '08081263000',
 '08081560665',
 '0825',
 '083',
 '0844',
 '08448350055',
 '08448714184',
 '0845',
 '08450542832',
 '084

## 2. Save, deploy, and score the Python Function <a id="deploy"></a>

In this section, you will learn how to save, deploy, and score the Python Function in the Watson Machine Learning (WML) repository.

- [2.1 Set up the environment](#setup)
- [2.2 Save and deploy the Python Function](#save)
- [2.3 Score data](#score)

### 2.1 Set up the environment <a id="setup"></a>

In this subsection, you will learn how to set up the Watson Machine learning (WML) service that is required in order to save, deploy, and score the `Python Function` in the Watson Machine learning (WML) repository.

#### Install the `watson-machine-learning-client` package from pypi
**Note:** `watson-machine-learning-client` documentation can be found <a href="http://wml-api-pyclient.mybluemix.net/" target="_blank" rel="noopener no referrer">here</a>.

In [6]:
!rm -rf $PIP_BUILD/watson-machine-learning-client

In [ ]:
!pip install --upgrade watson-machine-learning-client

Authenticate the Watson Machine Learning service on the IBM Cloud.

**Tip**: Authentication information (your credentials) can be found in the <a href="https://console.bluemix.net/docs/services/service_credentials.html#service_credentials" target="_blank" rel="noopener no referrer">Service credentials</a> tab of the service instance that you created on IBM the Cloud. <BR>If you cannot find the **instance_id** field in **Service Credentials**, click **New credential (+)** to generate new authentication information. 

**Action**: Enter your Watson Machine Learning service instance credentials in the following cell.

In [8]:
wml_credentials = {
    'url': 'https://ibm-watson-ml.mybluemix.net',
    'username': '---',
    'password': '---',
    'instance_id': '---'
}

In [9]:
# @hidden_cell

wml_credentials = {
  "instance_id": "d0755448-6eb4-425a-b35b-479be91ff2d5",
  "password": "bd202fa8-b3ca-4548-abd2-848828caa100",
  "url": "https://us-south.ml.cloud.ibm.com",
  "username": "d06414fa-5d62-4a18-aaaa-970cdccaa08b"
}

#### Import the `watson-machine-learning-client` module and authenticate the service instance.

In [10]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Now, let's instantiate a WatsonMachineLearningAPIClient object.

In [11]:
client = WatsonMachineLearningAPIClient(wml_credentials)

### 2.2 Save and deploy the Python Function <a id="save"></a>

In this subsection, you will learn how to save and deploy the `Python Function`.
First, store the `Python Function` with the meta data.

In [12]:
meta_data = {client.repository.FunctionMetaNames.NAME : 'Web scraping python function'}
function_details = client.repository.store_function(meta_props=meta_data, function=py_funct) # If the py_param dict is not empty, function=py_funct(py_param).

No RUNTIME_UID passed. Creating default runtime... SUCCESS

Successfully created default runtime with uid: a19d2ca3-4451-41f5-b919-1df0e1aa209b


Second, deploy the `Python Function`.

In [13]:
function_id = function_details['metadata']['guid']
function_deployment_details = client.deployments.create(artifact_uid=function_id, name='Web scraping python function deployment')



#######################################################################################

Synchronous deployment creation for uid: '899f96b0-8023-48fd-9819-c260fba91d7d' started

#######################################################################################


INITIALIZING
DEPLOY_IN_PROGRESS.
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='5720d008-f2f8-4db9-bd42-e2d5827f60fd'
------------------------------------------------------------------------------------------------




You can check the deployment details by running the following cell.

In [14]:
function_deployment_details

{'entity': {'deployable_asset': {'created_at': '2019-02-27T15:57:35.755Z',
   'guid': '899f96b0-8023-48fd-9819-c260fba91d7d',
   'name': 'Web scraping python function',
   'type': 'function',
   'url': 'https://us-south.ml.cloud.ibm.com/v4/functions/899f96b0-8023-48fd-9819-c260fba91d7d'},
  'deployed_version': {'created_at': '2019-02-27T15:57:35.755Z',
   'guid': '9f8d80d4-edcd-4587-8caa-9062f5072ef5',
   'url': 'https://us-south.ml.cloud.ibm.com/v4/functions/899f96b0-8023-48fd-9819-c260fba91d7d/revisions/9f8d80d4-edcd-4587-8caa-9062f5072ef5'},
  'description': 'Description of deployment',
  'model_type': 'n/a',
  'name': 'Web scraping python function deployment',
  'runtime_environment': 'python-3.5',
  'scoring_url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/d0755448-6eb4-425a-b35b-479be91ff2d5/deployments/5720d008-f2f8-4db9-bd42-e2d5827f60fd/online',
  'status': 'DEPLOY_SUCCESS',
  'status_details': {'status': 'DEPLOY_SUCCESS'},
  'type': 'online'},
 'metadata': {'created_

Please check if the deployment was successfully created by listing deployments.

In [15]:
client.deployments.list()

------------------------------------  ---------------------------------------  ------  --------------  ------------------------  ---------  -------------
GUID                                  NAME                                     TYPE    STATE           CREATED                   FRAMEWORK  ARTIFACT TYPE
5720d008-f2f8-4db9-bd42-e2d5827f60fd  Web scraping python function deployment  online  DEPLOY_SUCCESS  2019-02-27T15:57:36.924Z  n/a        function
------------------------------------  ---------------------------------------  ------  --------------  ------------------------  ---------  -------------


### 2.3 Score data <a id="score"></a>

In this subsection, you will learn how to score the deployed `Python Function` with a test data record.

First, create an online deployment endpoint.

In [16]:
function_deployment_endpoint_url = client.deployments.get_scoring_url(function_deployment_details)

In [17]:
function_deployment_endpoint_url

'https://us-south.ml.cloud.ibm.com/v3/wml_instances/d0755448-6eb4-425a-b35b-479be91ff2d5/deployments/5720d008-f2f8-4db9-bd42-e2d5827f60fd/online'

The following is the record that will be used for scoring.

In [18]:
payload = {
    'values': [ 
        'https://www.ibm.com/cloud/machine-learning' 
    ] 
}

In [19]:
client.deployments.score(function_deployment_endpoint_url, payload)['tokens']

['accelerate',
 'accelerating',
 'access',
 'accuracy',
 'actively',
 'advantage',
 'ai',
 'algorithms',
 'all',
 'an',
 'analytics',
 'analyze',
 'and',
 'anywhere',
 'apache',
 'apis',
 'applications',
 'apps',
 'architecture',
 'are',
 'around',
 'artificial',
 'as',
 'aspects',
 'assets',
 'at',
 'automate',
 'automated',
 'automates',
 'automatically',
 'be',
 'behavioral',
 'book',
 'build',
 'building',
 'business',
 'by',
 'can',
 'challenge',
 'check',
 'cloud',
 'collaboratively',
 'collect',
 'consultation',
 'content',
 'continuous',
 'continuously',
 'cost',
 'create',
 'creates',
 'critical',
 'cross',
 'customers',
 'dashboards',
 'data',
 'databases',
 'decentralize',
 'deep',
 'deliver',
 'demand',
 'deploy',
 'deployed',
 'deployment',
 'deployments',
 'design',
 'developers',
 'distribute',
 'distributed',
 'dl',
 'drive',
 'easily',
 'easy',
 'effective',
 'empowers',
 'enable',
 'enables',
 'end',
 'engineering',
 'enterprise',
 'environment',
 'environments',
 'ev

## 3. Summary and next steps <a id="summary"></a>

You successfully completed this notebook! 
 
You learned how to define a `Python Function`. Also, you learned how to save, deploy, and score the `Python Function` in the Watson Machine Learning (WML) repository. 

In the next step, in addition to tokenizing, a classification model trained with the `SMS spam` data set will be called in the `score()` function and perform scoring.
 
Check out our <a href="https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html" target="_blank" rel="noopener noreferrer">Online Documentation</a> for more samples, tutorials, documentation, how-tos, and blog posts.

### Citation

Almeida, T. A. and Hidalgo, J. M. G. (2012), [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection), Irvine, CA.

### Author

**Jihyoung Kim**, Ph.D., is a Data Scientist at IBM who strives to make data science easy for everyone through Watson Studio.

Copyright © 2019 IBM. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>